In [13]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [14]:
weights = None  # ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

# reshape last layer.
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model.load_state_dict(torch.load(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Weights\resnet18_torchvision\test43_epoch346.pth"))
# model.load_state_dict(torch.load(r"C:\Users\Noel\Documents\THESIS STUFF\PYTHON-THINGIES\Saved Model Parameters\resnet18_torchvision\test39_epoch348.pth"))
# Set model to evaluation mode and send to device
model.to(device).eval()

layers_of_interest = [name for name, _ in model.named_modules() if "conv" in name or "fc" in name]

print(layers_of_interest)


['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2', 'fc']


This cell works, but is technically wrong and would not 'fly' if the notebook was to be converted to an actual python script.

In [15]:
# layer_activations = {}

# def hook_wrapper(name: str):
#     def hook_fn(module: nn.Module, input: torch.Tensor, output: torch.Tensor) -> None:
#         layer_activations[name] = output
#     return hook_fn


# for name, layer in model.named_modules():
#     if name in layers_of_interest:
#         layer.register_forward_hook(hook_wrapper(name))

To combat this, I made the hooks into objects of a hook class, holding both the output and the hook function, thus creating dictionary entries 
of a key/value pair of name/Hook_Layer object.

In [16]:
class Hook_Layer():
    def __init__(self, layer) -> None:
        self.hook = layer.register_forward_hook(self.hook_fn)
        self.output = None

    def hook_fn(self, layer, input, output):
        self.output = output
    
    def __call__(self):
        return self.output

layer_activations = {}
for name, layer in model.named_modules():
    if name in layers_of_interest:
        layer_activations[name] = Hook_Layer(layer)

print(layer_activations.values())

dict_values([<__main__.Hook_Layer object at 0x00000183BEDA00A0>, <__main__.Hook_Layer object at 0x00000183BEDA38E0>, <__main__.Hook_Layer object at 0x00000183BEDA2E60>, <__main__.Hook_Layer object at 0x00000183BEDA2CE0>, <__main__.Hook_Layer object at 0x00000183BEDA2BF0>, <__main__.Hook_Layer object at 0x00000183BEDA2AA0>, <__main__.Hook_Layer object at 0x00000183BEDA2920>, <__main__.Hook_Layer object at 0x00000183BEDA2830>, <__main__.Hook_Layer object at 0x00000183BEDA3DF0>, <__main__.Hook_Layer object at 0x00000183BEDA32E0>, <__main__.Hook_Layer object at 0x00000183BEDA2F50>, <__main__.Hook_Layer object at 0x00000183BEDA1630>, <__main__.Hook_Layer object at 0x00000183BEDA33A0>, <__main__.Hook_Layer object at 0x00000183BEDA34F0>, <__main__.Hook_Layer object at 0x00000183BEDA3670>, <__main__.Hook_Layer object at 0x00000183BEDA3760>, <__main__.Hook_Layer object at 0x00000183B39FC550>, <__main__.Hook_Layer object at 0x00000183B39FFEB0>])


In [17]:
# Create a dataset class that extends ImageFolder while
# simultaneously returning a 3 way Tuple, instead of the
# original that contains 2 elements.
# For that reason we must define a new __getitem__ method.
class ImageFolderWithPaths(ImageFolder):
    """Dataset class extending ImageFolder dataset,
        returning Tuple.
        
        Returns:
                Tuple[img[torch.Tensor],
                      label[int],
                      path[str]]
        """
    def __getitem__(self, index: int):
        # Super the __getitem__ of base class
        img, label = super().__getitem__(index)
        # Extract the path of each image in the dataset
        path = self.imgs[index][0]
        # Return new tuple with 3 elements
        return (img, label, path)

    

In [18]:
batch_size = 64


transforms = T.Compose([T.Resize(224),
                        # T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize([0.5162, 0.4644, 0.3975],
                                    [0.2724, 0.2640, 0.2574])
                        ])

dataset = ImageFolderWithPaths(root=r"C:\Users\Noel\Documents\THESIS\Data\artbench-10-imagefolder-split\train",
                               transform=transforms)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print("Dataloader Initialized. Note that workers tend to take some time to \
initialize but speed up performance when loading.")
# ================================================================
data = []

with torch.no_grad():
    for images, labels, paths in tqdm(dataloader, total=len(dataloader)):
        # Send stuff to GPU if available.
        images = images.to(device)
        labels = labels.to(device)
        # Make Forward Pass.
        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        # path_list = []
        # for path in paths:
        #     path_list.append(path)

        for i, image in enumerate(images):
            private_dict = {}
            # Three entries regarding the image identification.
            # private_dict['path'] = path_list[i]
            private_dict['path'] = paths[i]
            private_dict['class_label'] = labels[i].item()
            private_dict['prediction'] = preds[i].item()
            # Iterate over all available layers.
            for key, hook_object in layer_activations.items():
                tensor_out = hook_object()  # .output  # modified from original script to accommodate objects 
                if key == 'fc':
                    # The array to store is a 32 by 10 array, each batch
                    output = torch.unbind(tensor_out, dim=0)
                else:
                    # The array will have a final shape of 32 by num_channels
                    # in specific layer
                    b, c, _, _ = tensor_out.shape
                    output = torch.unbind(tensor_out.view(b, c, -1).mean(2), dim=0)
                private_dict[key] = output[i].cpu().numpy()
            data.append(private_dict)

Dataloader Initialized. Note that workers tend to take some time to initialize but speed up performance when loading.


100%|██████████| 782/782 [12:57<00:00,  1.01it/s]


In [19]:
df = pd.DataFrame(data, copy=False)
# df.head()

In [20]:
df.to_parquet('test43_activations.parquet')

In [21]:
df = pd.read_parquet('test43_activations.parquet')
df.head()

,path,class_label,prediction,conv1,layer1.0.conv1,layer1.0.conv2,layer1.1.conv1,layer1.1.conv2,layer2.0.conv1,layer2.0.conv2,...,layer2.1.conv2,layer3.0.conv1,layer3.0.conv2,layer3.1.conv1,layer3.1.conv2,layer4.0.conv1,layer4.0.conv2,layer4.1.conv1,layer4.1.conv2,fc
0,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,0,"[-0.002602173, 0.0023542345, -0.0009352063, -0...","[-0.020409819, 0.0035067801, 0.0015911584, -0....","[-0.0050180135, -0.0041677076, 0.0026548335, -...","[-0.009137822, -0.022974238, -0.031220159, 0.0...","[-0.0012348989, -0.0026018713, -0.00071466615,...","[-0.01681667, -0.014908907, -0.01388878, -0.00...","[0.0059582554, -0.008185522, -0.0016718919, 0....",...,"[-0.0024276576, -0.0017781443, -0.00149582, -0...","[-0.002096928, -0.0049843974, -0.012551237, -0...","[-0.00039672552, 7.099305e-05, 0.002342132, -0...","[-0.009905416, -0.005991422, -0.00737815, 0.00...","[-0.0036616398, -0.0032120817, -0.0027941407, ...","[0.004260477, 0.0007153986, -0.003995311, -0.0...","[0.0020544836, -0.0042087603, -0.0009130098, -...","[-0.055628125, 0.006365579, -0.02327682, -0.00...","[0.0015513197, -0.005267969, -0.005100446, -0....","[3.1452694, -2.1385002, 0.043979976, 0.0959231..."
1,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,0,"[-0.0050166207, 0.0028267067, -0.000799118, -0...","[-0.021756314, 0.003405606, 0.0074080713, -0.0...","[-0.0075829616, -0.0038331905, -0.0007364194, ...","[-0.0026154262, -0.022285359, -0.03428321, 0.0...","[-0.0026339095, -0.0022114376, -0.001708517, -...","[-0.015480543, -0.015057802, -0.01055582, -0.0...","[0.005782616, -0.006907712, 7.587623e-05, 0.00...",...,"[-0.0023599088, -0.0033229133, -0.0017364791, ...","[-0.0024459483, -0.0009207275, -0.010160503, -...","[-0.004535223, -0.0028074353, 0.0021947962, -0...","[-0.009153635, -0.0049401745, -0.012273624, -0...","[-0.0033709626, -0.005904208, -0.0055196052, -...","[-0.0009999706, -0.008072821, -0.0050073215, -...","[-0.00052807824, -0.0035410344, -0.0010910128,...","[-0.045520287, 0.010214967, -0.011139404, -0.0...","[-0.0031815371, -0.0071409037, -0.004296401, 0...","[4.5233274, -2.3253276, 0.8490778, 0.2592919, ..."
2,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,3,"[0.0038523192, -0.0022469827, -0.0034269264, -...","[-0.01949254, 0.0047682067, 0.00471524, -0.019...","[-0.0068434323, -0.0040711947, -0.0030180872, ...","[-0.00022107257, -0.02334753, -0.03488163, 0.0...","[-0.003038024, -0.004398583, -0.002517813, -0....","[-0.016386623, -0.011239141, -0.0068872953, -0...","[0.008900666, -0.00816301, -0.0034607465, 0.00...",...,"[-0.0027881484, -0.0011914464, -0.0016503513, ...","[-0.0039676214, -0.0023938594, -0.005679932, -...","[-0.0024218205, -0.002857834, -0.0021098037, -...","[-0.0043306258, -0.007567052, -0.011723988, 0....","[-0.003541594, -0.002451755, -0.0036180709, -0...","[-0.0044040345, -0.003432324, -0.0036658535, 0...","[-0.0032970528, -0.0017908813, -0.0032233566, ...","[-0.051091593, 0.0046816785, -0.028427713, -0....","[-0.0067426926, -0.0051218527, -0.0053907954, ...","[-0.00055587105, -2.1791682, 0.27025473, 3.874..."
3,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,0,"[-0.0112717915, 0.0042773583, 0.005608209, 0.0...","[-0.019305026, 0.0059218733, 0.001578818, -0.0...","[-0.0047069495, -0.0035117073, 0.0001630498, -...","[-0.004830742, -0.020165522, -0.030705588, 0.0...","[-0.0019379597, -0.003069448, -0.0012110379, -...","[-0.017323406, -0.0112946, -0.011791457, -0.00...","[0.005908948, -0.0075597423, -0.0023350974, 0....",...,"[-0.0025527172, -0.0016888649, -0.0012635168, ...","[-0.006891447, -0.001933998, -0.013025512, -0....","[-0.005361996, -0.00088757277, -8.694305e-06, ...","[-0.0066956244, -0.0055859573, -0.0024006548, ...","[-0.002678157, -0.0033175815, -0.003776386, -0...","[0.0027688667, -0.002645061, -0.0014375815, -0...","[-0.0003056924, -0.00455963, 0.0022580023, -0....","[-0.04124376, 0.003812021, -0.009134978, -0.00...","[-0.0034850878, -0.008121065, -0.002973558, -0...","[4.56273